In [2]:
import saspy
sas = saspy.SASsession()

Using SAS Config named: oda
SAS Connection established. Subprocess id is 15824



### retain across entire table 

In [10]:
sas.submitLST(
f"""
data work.test;
    set work.class;
    
    /* create vars */
    length heaviest_names $100.;
    retain total_weight 0 max_weight . heaviest_names "";
            
    /* calculate using retained var*/
    total_weight = sum(total_weight, weight);
    max_weight = max(max_weight, weight);
    if weight = max_weight then heaviest_names = catx(', ', heaviest_names, name);

run;

proc print data=work.test;
run;
""")

Obs,Name,Sex,Age,Height,Weight,heaviest_names,total_weight,max_weight
1,Alice,F,13,56.5,84.0,Alice,84.0,84.0
2,Barbara,F,13,65.3,98.0,"Alice, Barbara",182.0,98.0
3,Carol,F,14,62.8,102.5,"Alice, Barbara, Carol",284.5,102.5
4,Jane,F,12,59.8,84.5,"Alice, Barbara, Carol",369.0,102.5
5,Janet,F,15,62.5,112.5,"Alice, Barbara, Carol, Janet",481.5,112.5
6,Joyce,F,11,51.3,50.5,"Alice, Barbara, Carol, Janet",532.0,112.5
7,Judy,F,14,64.3,90.0,"Alice, Barbara, Carol, Janet",622.0,112.5
8,Louise,F,12,56.3,77.0,"Alice, Barbara, Carol, Janet",699.0,112.5
9,Mary,F,15,66.5,112.0,"Alice, Barbara, Carol, Janet",811.0,112.5
10,Alfred,M,14,69.0,112.5,"Alice, Barbara, Carol, Janet, Alfred",923.5,112.5


### retain across by group to act as aggregation

In [19]:
sas.submitLST(
f"""
proc sort data=sashelp.cars out=work.cars;
    by type;

data work.test (keep=type max_mpg);
    set work.cars;
    by type;
    retain max_mpg;

    if first.type then do;
        max_mpg = 0;
    end;

    max_mpg = max(max_mpg, mpg_highway);

    if last.type then output;
run;

title;
proc print data=work.test;
run;

proc sql;
    select
        type,
        max(mpg_highway) as max_mpg
    from sashelp.cars
    group by 1
    order by 1
    ;
quit;
""")

### retain across by group to concatenate characters

In [32]:
sas.submitLST(
f"""
proc sort data=sashelp.class out=work.class;
    by sex name;

data work.test (keep=sex cat_names);
    set work.class;
    by sex;
    length cat_names $100;
    retain cat_names;

    if first.sex then do;
        cat_names = name;
    end;
    else do;
        cat_names = catx(", ",cat_names, name);
    end;

    if last.sex then output;
run;

title;
proc print data=work.test;
run;
""")
sas.sasdata("test","work").columnInfo()

Obs,Sex,cat_names
1,F,"Alice, Barbara, Carol, Jane, Janet, Joyce, Judy, Louise, Mary"
2,M,"Alfred, Henry, James, Jeffrey, John, Philip, Robert, Ronald, Thomas, William"


,Member,Num,Variable,Type,Len,Pos
0,WORK.TEST,1.0,Sex,Char,1.0,0.0
1,WORK.TEST,2.0,cat_names,Char,100.0,1.0


### sum statement

In [4]:
sas.submitLST(
f"""
data work.test;
    set sashelp.class;
    total_weight + weight;
run;
            
proc print data=work.test;
    var name weight total_weight;
run;
""")

Obs,Name,Weight,total_weight
1,Alfred,112.5,112.5
2,Alice,84.0,196.5
3,Barbara,98.0,294.5
4,Carol,102.5,397.0
5,Henry,102.5,499.5
6,James,83.0,582.5
7,Jane,84.5,667.0
8,Janet,112.5,779.5
9,Jeffrey,84.0,863.5
10,John,99.5,963.0


### others

In [11]:
sas.sasdata("cars","sashelp").head()

,Make,Model,Type,Origin,DriveTrain,MSRP,Invoice,EngineSize,Cylinders,Horsepower,MPG_City,MPG_Highway,Weight,Wheelbase,Length
0,Acura,MDX,SUV,Asia,All,36945.0,33337.0,3.5,6.0,265.0,17.0,23.0,4451.0,106.0,189.0
1,Acura,RSX Type S 2dr,Sedan,Asia,Front,23820.0,21761.0,2.0,4.0,200.0,24.0,31.0,2778.0,101.0,172.0
2,Acura,TSX 4dr,Sedan,Asia,Front,26990.0,24647.0,2.4,4.0,200.0,22.0,29.0,3230.0,105.0,183.0
3,Acura,TL 4dr,Sedan,Asia,Front,33195.0,30299.0,3.2,6.0,270.0,20.0,28.0,3575.0,108.0,186.0
4,Acura,3.5 RL 4dr,Sedan,Asia,Front,43755.0,39014.0,3.5,6.0,225.0,18.0,24.0,3880.0,115.0,197.0


In [13]:
your_data = "sashelp.cars"
group_var = "type"
order_var = "mpg_city"
var = "mpg_city"

sas.submitLST(
f"""
PROC SORT data={your_data} out=work.sorted_data;
   BY {group_var} {order_var};
RUN;

DATA new_data (KEEP={group_var} {var}_max {var}_lag {var}_cat count);
   SET sorted_data;
   BY {group_var};
   
   length {var}_cat $100;
   RETAIN {var}_max {var}_lag {var}_cat count;
   
   IF FIRST.{group_var} THEN DO;
      {var}_max = .;
      {var}_lead = .;
      {var}_cat = "";
      count = 0;
   END;
   
   IF {var} > {var}_max THEN {var}_max = {var};
   {var}_lag = LAG({var});
   {var}_cat = {var}_cat || ", " || PUT({var}, 8.2);
   count + 1;
   
   IF LAST.{group_var} THEN OUTPUT;
RUN;

PROC PRINT data=new_data;
run;
""", method="logandlist")


55                                                         The SAS System                         Saturday, May 18, 2024 10:22:00 AM

479        ods listing close;ods html5 (id=saspy_internal) file=_tomods1 options(bitmap_mode='inline') device=svg style=Plateau; ods
479      !  graphics on / outputfmt=png;
480        
481        
482        PROC SORT data=sashelp.cars out=work.sorted_data;
483           BY type mpg_city;
484        RUN;
485        
486        DATA new_data (KEEP=type mpg_city_max mpg_city_lag mpg_city_cat count);
487           SET sorted_data;
488           BY type;
489        
490           length mpg_city_cat $100;
491           RETAIN mpg_city_max mpg_city_lag mpg_city_cat count;
492        
493           IF FIRST.type THEN DO;
494              mpg_city_max = .;
495              mpg_city_lead = .;
496              mpg_city_cat = "";
497              count = 0;
498           END;
499        
500           IF mpg_city > mpg_city_max THEN mpg_city_max = mpg_city;
501 

Obs,Type,mpg_city_cat,mpg_city_max,mpg_city_lag,count
1,Hybrid,,60,59,3
2,SUV,,22,22,60
3,Sedan,,38,36,262
4,Sports,,26,24,49
5,Truck,,24,24,24
6,Wagon,,31,29,30
